In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import os
from tensorflow.keras import layers
import string

2025-11-08 09:54:18.260668: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762595658.440446      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762595658.493505      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
BATCH_SIZE = 128
NUM_HEADS = 8
NUM_BLOCKS = 6
EMBED_DIM = 256
DENSE_DIM = 1024
DROPOUT_RATE = 0.1
CHUNK_LENGTH = 150

In [3]:
df = pd.read_csv('/kaggle/input/the-bards-best-a-character-modeling-dataset/train.csv')

In [4]:
text = df.values[0][0]
text = re.sub(r'\s+', ' ', str(text)).strip()

In [5]:
import re

words = text.split()

# Count unique words
unique_words = set(words)
print(f"Total words: {len(words)}")
print(f"Unique words: {len(unique_words)}")


Total words: 182499
Unique words: 23841


In [6]:
def chunk_text_by_words(text, max_words, stride=None):
    words = text.split()
    if stride is None:
        stride = max_words // 2
    chunks = []
    for i in range(0, len(words) - max_words, stride):
        chunk = ' '.join(words[i:i + max_words])
        chunks.append(chunk)
    return chunks

In [7]:
chunks = chunk_text_by_words(text, CHUNK_LENGTH+1, 40)

In [8]:
print(np.shape(chunks))

(4559,)


In [9]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 13000
sequence_length = CHUNK_LENGTH+1

vectorizer = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
    standardize=custom_standardization
)

vectorizer.adapt(chunks)

I0000 00:00:1762595670.819286      39 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [10]:
vocab = vectorizer.get_vocabulary()
print("Total unique tokens in vocabulary:", len(vocab))

Total unique tokens in vocabulary: 12074


In [11]:
def make_dataset(chunks):
    tokens = vectorizer(chunks)
    tokens_inp = tokens[:,:CHUNK_LENGTH]
    tokens_out = tokens[:,1:]
    ds = tf.data.Dataset.from_tensor_slices((tokens_inp,tokens_out))
    ds = ds.batch(BATCH_SIZE)
    ds = ds.shuffle(1024).prefetch(16).cache()
    return ds

In [12]:
ds = make_dataset(chunks)

In [13]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, vocab_size, output_dim):
        super().__init__()
        self.positional_embedding = tf.keras.layers.Embedding(input_dim = sequence_length, output_dim = output_dim, mask_zero=False)
        self.token_embedding = tf.keras.layers.Embedding(input_dim = vocab_size, output_dim= output_dim, mask_zero=True)
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embedding(inputs)
        embedded_positions = self.positional_embedding(positions)
        return embedded_tokens + embedded_positions

In [14]:
class TransformerDecoder(tf.keras.layers.Layer):
    def __init__(self, num_heads, embed_dim, dense_dim, dropout_rate):
        super().__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                           key_dim=embed_dim//num_heads)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dense_proj = tf.keras.models.Sequential([
            tf.keras.layers.Dense(dense_dim, activation='gelu'),
            tf.keras.layers.Dense(embed_dim)
        ])
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-5)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
    def call(self, inputs):
        attn_out = self.attention(query=inputs,
                            key=inputs,
                            value=inputs,
                            use_causal_mask=True)
        norm1_out = self.layernorm1(attn_out+inputs)
        drop1_out = self.dropout1(norm1_out)
        dense_proj_out = self.dense_proj(drop1_out)
        norm2_out = self.layernorm2(drop1_out+dense_proj_out)
        drop2_out = self.dropout2(norm2_out)
        return drop2_out

In [15]:
inputs = tf.keras.layers.Input(shape=(None,))
embeddings = PositionalEmbedding(sequence_length, vocab_size, EMBED_DIM)(inputs)
x = embeddings
for layer in range(NUM_BLOCKS):
    x = TransformerDecoder(NUM_HEADS, EMBED_DIM, DENSE_DIM, DROPOUT_RATE)(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(vocab_size)(x)
transformer = tf.keras.models.Model(inputs, output)

In [16]:
transformer.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_embedding            │ (None, None, 256)      │     3,366,656 │
│ (PositionalEmbedding)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder             │ (None, None, 256)      │       789,760 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_1           │ (None, None, 256)      │       789,760 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_2           │ (None, None, 256)      │       789,760 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_3           │ (None, None, 256)      │       789,760 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_4           │ (None, None, 256)      │       789,760 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_5           │ (None, None, 256)      │       789,760 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, None, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, None, 13000)    │     3,341,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,446,216 (43.66 MB)

 Trainable params: 11,446,216 (43.66 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow.keras.backend as K

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)

def perplexity(y_true, y_pred):
    cross_entropy = K.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)
    return K.exp(K.mean(cross_entropy))

transformer.compile(loss = loss_fn,
                    metrics = ['accuracy', perplexity],
                    optimizer=opt)
transformer.fit(ds, epochs = 100)

Epoch 1/100


I0000 00:00:1762595698.008692      99 service.cc:148] XLA service 0x78b48c023260 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1762595698.009619      99 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
W0000 00:00:1762595699.280774      99 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
W0000 00:00:1762595699.280895      99 assert_op.cc:38] Ignoring Assert operator SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1762595700.291747      99 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1762595713.615183      99 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


13/36 ━━━━━━━━━━━━━━━━━━━━ 4s 203ms/step - accuracy: 0.0169 - loss: 8.8354 - perplexity: 8188.5547

W0000 00:00:1762595717.369800      98 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
W0000 00:00:1762595717.369864      98 assert_op.cc:38] Ignoring Assert operator SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


36/36 ━━━━━━━━━━━━━━━━━━━━ 59s 568ms/step - accuracy: 0.0223 - loss: 8.0754 - perplexity: 4949.9932
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 200ms/step - accuracy: 0.0291 - loss: 6.8148 - perplexity: 913.4657
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 201ms/step - accuracy: 0.0299 - loss: 6.8017 - perplexity: 901.4036
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 201ms/step - accuracy: 0.0300 - loss: 6.7962 - perplexity: 896.3240
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 201ms/step - accuracy: 0.0301 - loss: 6.7945 - perplexity: 894.7186
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 201ms/step - accuracy: 0.0296 - loss: 6.7915 - perplexity: 891.9363
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 201ms/step - accuracy: 0.0298 - loss: 6.7896 - perplexity: 890.3498
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 201ms/step - accuracy: 0.0299 - loss: 6.7879 - perplexity: 888.7051
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 201ms/step - accuracy: 0.0299 - loss: 6.7870 - perplexity: 887.9250
Epoch 10/100
36/36 ━

In [ ]:
def generate_text(prompt, max_length=50):
    for _ in range(max_length):
        tokenized = vectorizer([prompt])
        preds = transformer(tokenized)
        next_id = tf.argmax(preds[0, -1, :]).numpy()
        next_word = vectorizer.get_vocabulary()[next_id]
        prompt += " " + next_word
        if next_id == 0:
            break
    return prompt

print(generate_text("my"))